# Projeto de Algoritmo para Busca de Canais Livres de Radiodifusão

## Importação das Bibliotecas

In [2]:
import xml.etree.ElementTree as ET
import requests
import pandas as pd
import geopandas as gpd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import shapely.geometry
from geopy.distance import geodesic
import plotly.graph_objects as go
from shapely.geometry import Point
import folium
import leafmap.foliumap as leafmap

import sys
sys.path.insert(0, r"/Users/hygson/Documents/3-Recursos/Python/Funcoes")

import relevo as rel
import propagacao as prop
import radiodifusao as rad

## Dados do Canal a Ser Incluído

In [3]:
# Coordenadas geográficas do canal
latitude_gms = "21° 52' 3.36'' S"  
longitude_gms = "45° 37' 1.02'' W"

# Raio de busca, em km
raio_busca = 150

# Classe do canal
classe = 'C'

# ERP máxima, em kW
erp_max = 0.08

# Diagrama da antena (a cada 10 graus)
Er = np.ones(36)

# ERP por azimute (a cada 10 graus)
erp_az = list(erp_max * Er)

# HCI, em metros
hci = 150

# Altura do receptor
hrx = 1.5

# Número do canal
canal = 47

# Frequência central do canal, em MHz
f = 671

In [6]:
# Coordenadas em graus decimais
latitude_decimal = rad.latitude_gms_para_decimal(latitude_gms)
longitude_decimal = rad.longitude_gms_para_decimal(longitude_gms)

# Canais de TV em UHF
canais_tv_uhf = [canal for canal in range(14, 52)]

# Relação de Proteção (RP)
rp_cocanal = 19
rp_canal_adjacente = -36

# Intensidade de Campo no Contorno Protegido (TV UHF)
campo_contorno_protegido = 51

# Intensidade de Campo no Contorno Interferente Cocanal
campo_contorno_interferente_cocanal = campo_contorno_protegido - rp_cocanal


## Funções

In [7]:
def ocupacao_canal(status):
    if 'C0' in status:
        return 'Canal Vago'
    else:
        return 'Canal Ocupado'


def idt_servico(servico):
    if servico == 'FM'.strip():
        return 'FM'
    if servico == 'RTRFM'.strip():
        return 'RTR'
    elif servico in ['RTVD'.strip(), 'GTVD'.strip(), 'PBTVD'.strip()]:
        return 'TVD'
    elif servico == 'TVA'.strip():
        return 'TVA'
    elif servico == 'TV'.strip():
        return 'TV'
    elif servico == 'RTV'.strip():
        return 'RTV'
    elif servico == 'OM'.strip():
        return 'OM'
    else:
        return 'verificar'


def subfaixa_tv(canal):
    if 2 <= canal <= 6:
        return "VHF Baixo"
    elif 7 <= canal <= 13:
        return "VHF Alto"
    elif 14 <= canal <= 51:
        return "UHF"
    else:
        return "NA"


def operacao(ocupacao):
    if ocupacao == 'Canal Ocupado':
        return 'Sim'
    elif ocupacao == 'Canal Vago':
        return 'Não'
    else:
        return 'Desconhecido'


def descricao_status(status):
    if 'C0' in status:
        return 'Canal Vago'
    elif 'C1' in status:
        return 'Canal Outorgado - Aguardando Ato de RF'
    elif 'C2' in status:
        return 'Canal Outorgado - Aguardando Dados da Estação'
    elif 'C3' in status:
        return 'Canal Outorgado - Aguardando Licenciamento'
    elif 'C4' in status:
        return 'Canal Licenciado'
    elif 'C5' in status:
        return 'Canal Pendente de Outorga'
    elif 'C7' in status:
        return 'Aguardando Ato de RF'
    elif 'C8' in status:
        return 'Aguardando Dados do Contrato'
    elif 'C98' in status:
        return 'Canal TVA'
    elif 'C99' in status:
        return 'Canal Suspenso'
    else:
        return 'Não definido'


def dmax_cp(classe, canal=None):

    # Identifica a distância máxima (dmax) ao contorno protegido (cp),
    # com base no número e na classe do canal. Função válida apenas para
    # canais de FM e de TV digital.

    canal = int(canal)

    if classe.lower() in ['e', 'especial']:
        if canal <= 13:
            dmax = 65.6
        elif canal <= 46:
            dmax = 58.0
        elif canal >= 47:
            dmax = 58.0
    elif classe == 'A':
        if canal <= 13:
            dmax = 47.9
        elif canal >= 14:
            dmax = 42.5
    elif classe == 'B':
        if canal <= 13:
            dmax = 32.3
        elif canal >= 14:
            dmax = 29.1
    elif classe == 'C':
        if canal == None:
            dmax = 7.5
        else:
            if canal <= 13:
                dmax = 20.2
            elif canal >= 14 and canal <= 51:
                dmax = 18.1
            else:
                dmax = 7.5
    elif classe == 'E1':
        dmax = 78.5
    elif classe == 'E2':
        dmax = 67.5
    elif classe == 'E3':
        dmax = 54.5
    elif classe == 'A1':
        dmax = 38.5
    elif classe == 'A2':
        dmax = 35.0
    elif classe == 'A3':
        dmax = 30.0
    elif classe == 'A4':
        dmax = 24.0
    elif classe == 'B1':
        dmax = 16.5
    elif classe == 'B2':
        dmax = 12.5

    return dmax


def circulo_gdf(latitude, longitude, raio):

    # Gera o shapefile de um círculo com centro em (latitude, longitude) e raio em metros.
    # Latitude e longitude em graus, raio em km

    centro_circulo = shapely.geometry.Point(longitude, latitude)
    gdf_centro_circulo = gpd.GeoDataFrame(
        geometry=[centro_circulo], crs='EPSG:4674')
    centro_circulo_utm = gdf_centro_circulo.to_crs(epsg=5880)
    circulo_utm = centro_circulo_utm.buffer(raio * 10**3)  # raio em metros
    circulo = circulo_utm.to_crs('EPSG:4674')

    return gpd.GeoDataFrame(geometry=circulo)


def circulo(latitude, longitude, raio):

    # Gera o círculo com centro em (latitude, longitude) e raio em metros.
    # Latitude e longitude em graus, raio em km

    centro_circulo = shapely.geometry.Point(longitude, latitude)
    gdf_centro_circulo = gpd.GeoDataFrame(
        geometry=[centro_circulo], crs='EPSG:4674')
    centro_circulo_utm = gdf_centro_circulo.to_crs(epsg=5880)
    circulo_utm = centro_circulo_utm.buffer(raio * 10**3)  # raio em metros
    circulo = circulo_utm.to_crs('EPSG:4674')

    return circulo[0]

## Importação dos Canais de TV do Plano Básico

### Canais de TV e FM

In [ ]:
arquivo_xml = r"/Users/hygson/Documents/3-Recursos/Dados/Radiodifusao/Canais/plano_basicoTVFM.xml"

tree =  ET.parse(arquivo_xml)
root = tree.getroot()

ids_canal = []
IdtPlanoBasicos = []
ufs = []
CodMunicipios = []
municipios = []
canais = []
frequencias = []
decalagens = []
classes = []
servicos = []
carateres = []
finalidades = []
statuss = []
entidades = []
cnpjs = []
latitudesGMS = []
longitudesGMS = []
latitudes = []
longitudes = []
erps = []
alturas = []
limitacoes = []
observacoes = []
fistels = []

for node in root.findall('row'): 
    ids_canal.append(node.get("id"))
    IdtPlanoBasicos.append(node.get("IdtPlanoBasico"))
    ufs.append(node.attrib.get("UF"))
    CodMunicipios.append(node.get("CodMunicipio"))
    municipios.append(node.get("Municipio"))
    canais.append(node.get("Canal"))
    frequencias.append(node.get("Frequência"))
    decalagens.append(node.get("Decalagem"))
    classes.append(node.get("Classe"))
    servicos.append(node.get("Servico"))
    carateres.append(node.get("Carater"))
    finalidades.append(node.get("Finalidade"))
    statuss.append(node.get("Status"))
    entidades.append(node.get("Entidade"))
    cnpjs.append(node.get("CNPJ"))
    latitudesGMS.append(node.get("LatitudeGMS"))
    longitudesGMS.append(node.get("LongitudeGMS"))
    latitudes.append(node.get("Latitude"))
    longitudes.append(node.get("Longitude"))
    erps.append(node.get("ERP"))
    alturas.append(node.get("Altura"))
    limitacoes.append(node.get("Limitacoes"))
    observacoes.append(node.get("Observacoes"))
    fistels.append(node.get("Fistel"))

dados = {'id': ids_canal,
         'IdtPlanoBasico': IdtPlanoBasicos,
         'UF': ufs,
         'CodMunicipio': CodMunicipios,
         'Municipio': municipios,
         'Canal': canais,
         'Frequência': frequencias,
         'Decalagem': decalagens,
         'Classe': classes,
         'Servico': servicos,
         'Carater': carateres,
         'Finalidade': finalidades,
         'Status': statuss,
         'Entidade': entidades,
         'CNPJ': cnpjs,
         'latitudeGMS': latitudesGMS,
         'longitudeGMS': longitudesGMS,
         'Latitude': latitudes,
         'Longitude': longitudes,
         'ERP': erps,
         'Altura': alturas,
         'Limitacoes': limitacoes,
         'Observacoes': observacoes,
         'Fistel': fistels}

# Canais de TV e FM
canais_tv_fm = pd.DataFrame(dados)
canais_tv_fm = canais_tv_fm.apply(lambda x: x.str.replace(',','.'))
canais_tv_fm[["Canal", "Frequência", "Latitude", "Longitude", "ERP", "Altura"]] = canais_tv_fm[["Canal", "Frequência", "Latitude", "Longitude", "ERP", "Altura"]].apply(pd.to_numeric)

# Modificação do tipo de algumas colunas e substituição dos valores em branco (NaN)
canais_tv_fm['Decalagem'] = canais_tv_fm['Decalagem'].replace(np.nan, 'Nenhuma')
canais_tv_fm['Classe'] = canais_tv_fm['Classe'].replace(np.nan, 'Não definida')
canais_tv_fm['Entidade'] = canais_tv_fm['Entidade'].replace(np.nan, 'Nenhuma')
canais_tv_fm['Tipo de Servico'] = canais_tv_fm['Servico'].apply(idt_servico)
canais_tv_fm['Subfaixa de TV'] = canais_tv_fm['Canal'].apply(subfaixa_tv)
canais_tv_fm['Ocupacao'] = canais_tv_fm['Status'].apply(ocupacao_canal)
canais_tv_fm['Descricao do Status'] = canais_tv_fm['Status'].apply(descricao_status)

### Canais de TV

In [ ]:
servicos_tv = ['TV', 'RTV', 'RTVD', 'GTVD', 'PBTVD']
filtro_tv = canais_tv_fm['Servico'].isin(servicos_tv)
filtro_classe_cadastrada = canais_tv_fm['Classe'] != ''
canais_tv = canais_tv_fm[filtro_tv & filtro_classe_cadastrada]
canais_tv = canais_tv.reset_index(drop=True)
canais_tv.loc[:, 'dmax_cp'] = np.vectorize(dmax_cp)(canais_tv['Classe'], canais_tv['Canal'])

### Canais de TV digital

In [ ]:
servicos_tv_digital = ['RTVD', 'GTVD', 'PBTVD']
filtro_tv_digital = canais_tv['Servico'].isin(servicos_tv_digital)
canais_tv_digital = canais_tv[filtro_tv_digital]
canais_tv_digital = canais_tv_digital.reset_index(drop=True)

In [ ]:
colunas_principais = ['id', 'UF', 'CodMunicipio', 'Municipio', 'Canal',
                      'Frequência', 'Classe', 'Servico', 'Carater', 'Finalidade',
                      'Status', 'Descricao do Status', 'Entidade', 'CNPJ', 'Latitude', 
                      'Longitude', 'ERP', 'Altura', 'Fistel', 'Tipo de Servico', 
                      'Subfaixa de TV', 'Ocupacao', 'dmax_cp']

In [ ]:
canais_tv_digital = canais_tv_digital[colunas_principais]

## Busca de Canais para Inclusão no Plano Básico

In [ ]:
# Filtrar os cocanais

filtro_cocanais = canais_tv['Canal'] == canal
cocanais_tv = canais_tv[filtro_cocanais]

# Calcular a distância do canal a ser incluído a cada canal de TV existente

def distancia(lat2, lon2):
    return rel.distancia(latitude_decimal, longitude_decimal, lat2, lon2)

cocanais_tv.loc[:, 'Distancia'] = np.vectorize(distancia)(cocanais_tv['Latitude'], cocanais_tv['Longitude'])

# Filtrar os cocanais distantes até o raio de busca

filtro_distancia = cocanais_tv['Distancia'] <= raio_busca
cocanais_tv = cocanais_tv[filtro_distancia]
cocanais_tv = cocanais_tv.reset_index(drop=True)

# Calcular os azimutes entre o canal a ser incluído e cada canal de TV existente

def azimute_1_2(lat2, lon2):
    return rel.azimute(latitude_decimal, longitude_decimal, lat2, lon2)

def azimute_2_1(lat2, lon2):
    return rel.azimute(lat2, lon2, latitude_decimal, longitude_decimal)

cocanais_tv.loc[:, 'Azimute 1-2'] = np.vectorize(azimute_1_2)(cocanais_tv['Latitude'], cocanais_tv['Longitude'])
cocanais_tv.loc[:, 'Azimute 2-1'] = np.vectorize(azimute_2_1)(cocanais_tv['Latitude'], cocanais_tv['Longitude'])

# Calcular as HNMT entre o canal a ser incluído e cada canal de TV existente

arquivo, matriz = rel.abrir_raster()

def hnmt_1_2(azimute_1_2):
    return rel.hnmt(arquivo, matriz, hci, latitude_decimal, longitude_decimal, azimute_1_2)

def hnmt_2_1(lat, lon, hci, azimute_2_1):
    return rel.hnmt(arquivo, matriz, hci, lat, lon, azimute_2_1)

cocanais_tv.loc[:, 'HNMT 1-2'] = cocanais_tv['Azimute 1-2'].apply(hnmt_1_2)
cocanais_tv.loc[:, 'HNMT 2-1'] = np.vectorize(hnmt_2_1)(cocanais_tv['Latitude'], cocanais_tv['Longitude'], cocanais_tv['Altura'], cocanais_tv['Azimute 2-1'])

# Se der erro, por causa de HNMT muito negativo, usar a linha abaixo:

# Se der erro, por causa de HNMT muito negativo, usar a linha abaixo:
# canais_tv = canais_tv[canais_tv['HNMT'] > 10]

# Calcular as distâncias ao contorno interferente cocanal

def distancia_contorno_interferente_1_2(hnmt_1_2):
    return prop.distancia_p1546(f, hnmt_1_2, erp, campo_contorno_interferente_cocanal)

cocanais_tv['dist CI 1-2'] = cocanais_tv['HNMT 1-2'].apply(distancia_contorno_interferente_1_2)

def distancia_contorno_interferente_2_1(erp, hnmt):
    return prop.distancia_p1546(f, hnmt, erp, campo_contorno_interferente_cocanal)

cocanais_tv.loc[:, 'dist CI 2-1'] = np.vectorize(distancia_contorno_interferente_2_1)(cocanais_tv['ERP'], 
                                                                                      cocanais_tv['HNMT 2-1'])

# Calcular as distâncias aos contornos protegidos cocanal

def distancia_contorno_protegido_1_2(hnmt_1_2):
    return prop.distancia_p1546(f, hnmt_1_2, erp, campo_contorno_protegido)

cocanais_tv['dist CP 1-2'] = cocanais_tv['HNMT 1-2'].apply(distancia_contorno_protegido_1_2)

def distancia_contorno_protegido_2_1(erp, hnmt_2_1):
    return prop.distancia_p1546(f, hnmt_2_1, erp, campo_contorno_protegido)

cocanais_tv.loc[:, 'dist CP 2-1'] = np.vectorize(distancia_contorno_protegido_2_1)(cocanais_tv['ERP'], 
                                                                                   cocanais_tv['HNMT 2-1'])

# Verificar a relação de proteção contra interferência

# Verificar a condição d_ci > d - d_cp, ou seja, se a distância do contorno interferente do canal 
# a ser incluído é maior do que a diferença entre a distância desse canal a cada um dos cocanais
# até o raio de busca e a distância ao contorno protegido desses canais.

# Se d_ci > d - d_cp ==> interferência
# Se d_ci <= d - d_cp ==> sem interferência

def verifica_relacao_protecao(d_ci, d, d_cp):
    if d_ci > d - d_cp:
        return "RP não atendida"
    else:
        return "RP atendida"

cocanais_tv.loc[:, 'Relação de Proteção 1-2'] = np.vectorize(verifica_relacao_protecao)(cocanais_tv['dist CI 1-2'], 
                                                                                        cocanais_tv['Distancia'], 
                                                                                        cocanais_tv['dist CP 2-1'])

cocanais_tv.loc[:, 'Relação de Proteção 2-1'] = np.vectorize(verifica_relacao_protecao)(cocanais_tv['dist CI 2-1'], 
                                                                                        cocanais_tv['Distancia'], 
                                                                                        cocanais_tv['dist CP 1-2'])

In [ ]:
colunas_interesse = ['UF', 'CodMunicipio', 'Municipio', 'Canal', 'Frequência', 'Classe', 
                     'Servico', 'Status', 'Descricao do Status', 'Entidade', 'Latitude', 
                     'Longitude', 'Distancia', 'dmax_cp', 'dist CP 1-2', 'dist CI 1-2', 
                     'dist CP 2-1', 'dist CI 2-1', 'Relação de Proteção 1-2', 'Relação de Proteção 2-1']
df_cocanais_tv = cocanais_tv.copy()
df_cocanais_tv = df_cocanais_tv[colunas_interesse]

In [ ]:
df_cocanais_tv

In [ ]:
cocanais_tv_rp_1_2_nao_atendida = cocanais_tv[cocanais_tv['Relação de Proteção 1-2'] == 'RP não atendida']
cocanais_tv_rp_1_2_nao_atendida[colunas_interesse]

In [ ]:
cocanais_tv_rp_1_2_atendida = cocanais_tv[cocanais_tv['Relação de Proteção 1-2'] == 'RP atendida']
cocanais_tv_rp_1_2_atendida[colunas_interesse]

In [ ]:
filtro_municipio = "São João da Ponte"
df = canais_tv[canais_tv['Municipio'] == filtro_municipio]
df = df[['Canal', 'Classe', 'Status', 'Entidade']]
df = df.reset_index(drop=True)
df

In [ ]:
prop.distancia_p1546(470, 500, 80, 32)

### Canais de Rede

In [ ]:
url = "https://www.in.gov.br/web/dou/-/portaria-mcom-n-12.331-de-26-de-fevereiro-de-2024-545104976"
lista_tabelas = pd.read_html(url)
tabelas = lista_tabelas[:-1]
tabela_1 = tabelas[0].iloc[1:]
tabela_1.columns = [0, 1, 2, 3]
tabelas[0] = tabela_1
df = pd.concat(tabelas)
df.columns = ['UF', 'Região', 'Entidade', 'Canal']

In [ ]:
df.head()

In [ ]:
canais_tv

In [ ]:
canais_tv_com_interferencia_1_2

In [ ]:
canais_tv_fm.shape